In [14]:
import string
import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import metrics
from imblearn.metrics import classification_report_imbalanced
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.multioutput import ClassifierChain

In [15]:
df = pd.read_csv('DataMerged.csv', sep=',',index_col='ID')
print(df.shape)
df.head()

(1690, 23)


,comment_content,Giao hang,Dong goi,Tieng click chuot,Phu kien,Gia ban,Thiet ke,Chat luong san pham,Pin,Cap ket noi,...,Nut power,Con lan,Nut ben trai,Nut ben phai,Do nhay,Phi ship,Ket noi,Tem bao hanh,Den,Che do sac
ID,,,,,,,,,,,,,,,,,,,,,
0,"Chuột nhạy, xài mượt, êm. Giá cũng OK. Sản phẩ...",1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,Thấy có vài bạn nói bị vênh này kia nhưng mình...,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
2,Khi mới nhận được sản phẩm thì chuột đã có dấu...,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
3,This is the best cheap wireless mouse I've eve...,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,TIKI cho hỏi chuột bị 1 vạch gờ lên do hoàn th...,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
df.drop_duplicates(subset ="comment_content", inplace = True)
df['comment_content'].replace('', np.nan, inplace=True)
df.dropna(subset=['comment_content'], inplace=True)
print(df.shape)

(1323, 23)


In [17]:
df.sum(numeric_only=True)

Giao hang              321
Dong goi               266
Tieng click chuot       85
Phu kien                28
Gia ban                131
Thiet ke               300
Chat luong san pham    931
Pin                     89
Cap ket noi              9
Cham soc khach hang    113
Bao hanh                 3
Bluetooth                5
Nut power               16
Con lan                 40
Nut ben trai            11
Nut ben phai             8
Do nhay                254
Phi ship                 7
Ket noi                 57
Tem bao hanh            46
Den                     33
Che do sac               1
dtype: int64

In [18]:
list_sum = list(df.sum(numeric_only=True))
idx = [i for i in range(len(list_sum)) if list_sum[i] < 50]
drop_column = [list(df)[i+1] for i in idx]
df = df.drop(drop_column, axis =1)
print(list(df))

['comment_content', 'Giao hang', 'Dong goi', 'Tieng click chuot', 'Gia ban', 'Thiet ke', 'Chat luong san pham', 'Pin', 'Cham soc khach hang', 'Do nhay', 'Ket noi']


In [19]:

#Convert all to lower case
df.comment_content = df.comment_content.apply(lambda x: x.lower())
#Remove punctuation, number, Emojify
df.comment_content = df.comment_content.str.replace(r"""[^a-z\s
                                    \à\á\ạ\ả\ã\â\ầ\ấ\ậ\ẩ\ẫ\ă\ằ\ắ\ặ\ẳ\ẵ
                                    \è\é\ẹ\ẻ\ẽ\ê\ề\ế\ệ\ể\ễ
                                    \ì\í\ị\ỉ\ĩ
                                    \ò\ó\ọ\ỏ\õ\ô\ồ\ố\ộ\ổ\ỗ\ơ\ờ\ớ\ợ\ở\ỡ
                                    \ù\ú\ụ\ủ\ũ\ư\ừ\ứ\ự\ử\ữ
                                    \ỳ\ý\ỵ\ỷ\ỹ
                                    \đ]""", '')
df.head()

,comment_content,Giao hang,Dong goi,Tieng click chuot,Gia ban,Thiet ke,Chat luong san pham,Pin,Cham soc khach hang,Do nhay,Ket noi
ID,,,,,,,,,,,
0,chuột nhạy xài mượt êm giá cũng ok sản phẩm gi...,1,0,0,0,0,0,0,0,1,0
1,thấy có vài bạn nói bị vênh này kia nhưng mình...,0,0,0,0,0,1,0,0,1,0
2,khi mới nhận được sản phẩm thì chuột đã có dấu...,0,0,0,0,0,1,0,1,1,0
3,this is the best cheap wireless mouse ive ever...,0,0,0,1,0,1,0,0,0,0
4,tiki cho hỏi chuột bị vạch gờ lên do hoàn thi...,0,0,0,0,0,1,0,0,0,0


In [20]:
patterns = {
    '[àáảãạăắằẵặẳâầấậẫẩ]': 'a',
    '[đ]': 'd',
    '[èéẻẽẹêềếểễệ]': 'e',
    '[ìíỉĩị]': 'i',
    '[òóỏõọôồốổỗộơờớởỡợ]': 'o',
    '[ùúủũụưừứửữự]': 'u',
    '[ỳýỷỹỵ]': 'y'
}

def toASCII(text):
    output = text
    for regex, replace in patterns.items():
        output = re.sub(regex, replace, output)
    return output

df_ascii = df.copy()
df_ascii.comment_content = df_ascii.comment_content.apply(lambda x: toASCII(x))
df_ascii.head()

,comment_content,Giao hang,Dong goi,Tieng click chuot,Gia ban,Thiet ke,Chat luong san pham,Pin,Cham soc khach hang,Do nhay,Ket noi
ID,,,,,,,,,,,
0,chuot nhay xai muot em gia cung ok san pham gi...,1,0,0,0,0,0,0,0,1,0
1,thay co vai ban noi bi venh nay kia nhung minh...,0,0,0,0,0,1,0,0,1,0
2,khi moi nhan duoc san pham thi chuot da co dau...,0,0,0,0,0,1,0,1,1,0
3,this is the best cheap wireless mouse ive ever...,0,0,0,1,0,1,0,0,0,0
4,tiki cho hoi chuot bi vach go len do hoan thi...,0,0,0,0,0,1,0,0,0,0


In [21]:
frames = [df, df_ascii]
df = pd.concat(frames)
print(df.shape)

(2646, 11)


In [22]:
target = list(df)[1:]
vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1,3))
vectorizer.fit(df['comment_content'])
for i in range(len(target)):
    X_train, X_test, y_train, y_test = train_test_split(df['comment_content'], df[target[i]], test_size=0.25,
                                                        stratify=df[target[i]], random_state=123456)
    vectors_train = vectorizer.transform(X_train)
    vectors_test = vectorizer.transform(X_test)
    clf = GaussianNB()
    #clf = AdaBoostClassifier(DecisionTreeClassifier(),n_estimators=10, random_state=123456)
    clf.fit(vectors_train.toarray(), y_train)
    predicted = clf.predict(vectors_test.toarray())
    accuracy = metrics.f1_score(y_test, predicted)
    print(target[i],': ',accuracy)

Giao hang :  0.7684210526315789
Dong goi :  0.7375415282392027
Tieng click chuot :  0.6213592233009709
Gia ban :  0.6982248520710059
Thiet ke :  0.7927927927927928
Chat luong san pham :  0.9065817409766453
Pin :  0.5471698113207547
Cham soc khach hang :  0.6979865771812079
Do nhay :  0.7167235494880546
Ket noi :  0.6493506493506493


In [24]:
X_train, X_test, y_train, y_test = train_test_split(df['comment_content'], df[target[:]],
                                                    stratify=df[target[:]], test_size=0.25, random_state=123456)
vectors_train = vectorizer.transform(X_train)
vectors_test = vectorizer.transform(X_test)
#clf = BinaryRelevance(AdaBoostClassifier(GaussianNB(),n_estimators=10, random_state=123456))
clf = BinaryRelevance(GaussianNB())
clf.fit(vectors_train, y_train)
predicted = clf.predict(vectors_test)
print('hamming loss: ',metrics.hamming_loss(y_test, predicted))
print('jaccard similarity_ core: ', metrics.jaccard_similarity_score(y_test, predicted))
print('micro averaging F1-Score: ', metrics.f1_score(y_test, predicted, average="micro"))
print('macro averaging F1-Score: ', metrics.f1_score(y_test, predicted, average="macro"))

hamming loss:  0.07643504531722055
jaccard similarity_ core:  0.8135298278425168
micro averaging F1-Score:  0.8147877013177159
macro averaging F1-Score:  0.760582620066842


In [26]:
#clf = BinaryRelevance(AdaBoostClassifier(DecisionTreeClassifier(),n_estimators=10, random_state=123456))
clf = BinaryRelevance(DecisionTreeClassifier())
clf.fit(vectors_train, y_train)
predicted = clf.predict(vectors_test)
print('hamming loss: ',metrics.hamming_loss(y_test, predicted))
print('jaccard similarity_ core: ', metrics.jaccard_similarity_score(y_test, predicted))
print('micro averaging F1-Score: ', metrics.f1_score(y_test, predicted, average="micro"))
print('macro averaging F1-Score: ', metrics.f1_score(y_test, predicted, average="macro"))

hamming loss:  0.0297583081570997
jaccard similarity_ core:  0.8806502661487554
micro averaging F1-Score:  0.9212944466640032
macro averaging F1-Score:  0.9176822778873444


In [40]:
#chains = [ClassifierChain(AdaBoostClassifier(DecisionTreeClassifier(),n_estimators=10, random_state=123456),
#                          order='random', random_state=i) for i in range(100)]
chains = [ClassifierChain(DecisionTreeClassifier(), order='random', random_state=i) for i in range(100)]
for chain in chains:
    chain.fit(vectors_train, y_train)

In [41]:
Y_pred_chains = np.array([chain.predict(vectors_test) for chain in chains])
chain_hamming_loss = [metrics.hamming_loss(y_test, Y_pred_chain) for Y_pred_chain in Y_pred_chains]
chain_jaccard_scores = [metrics.jaccard_similarity_score(y_test, Y_pred_chain) for Y_pred_chain in Y_pred_chains]
chain_micro_f1_score = [metrics.f1_score(y_test, Y_pred_chain, average="micro") for Y_pred_chain in Y_pred_chains]
chain_macro_f1_score = [metrics.f1_score(y_test, Y_pred_chain, average="macro") for Y_pred_chain in Y_pred_chains]

In [42]:
idx = chain_macro_f1_score.index(max(chain_macro_f1_score))
print('hamming loss: ', chain_hamming_loss[idx])
print('jaccard similarity_ core: ', chain_jaccard_scores[idx])
print('micro averaging F1-Score: ', chain_micro_f1_score[idx])
print('macro averaging F1-Score: ', chain_macro_f1_score[idx])
order = [target[i] for i in chains[idx].order_]
print(order)

hamming loss:  0.030060422960725076
jaccard similarity_ core:  0.8796971658754136
micro averaging F1-Score:  0.9204318272690923
macro averaging F1-Score:  0.9201377938741263
['Pin', 'Dong goi', 'Do nhay', 'Ket noi', 'Gia ban', 'Tieng click chuot', 'Giao hang', 'Cham soc khach hang', 'Thiet ke', 'Chat luong san pham']
